In [1]:
import pandas as pd
import os
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

In [2]:
df=pd.read_stata("d://r3.dta")
df.drop(["index","year"],axis=1,inplace= True)

In [12]:
# df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())
CPCleanedUnique=list(df.CPCleaned.unique())

In [13]:
def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp
        
def getOriginalQ(cleanedQ):
    global df
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

def groupSent(sentences, threshold=0.75):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences,show_progress_bar=False)
    clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
    clustering.fit(embeddings)
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(sentences[sentence_id])
    return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

# with open("grpdQ.txt","r") as txt:
#     grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def collapse(qcleaned, country):
    global df
    return df[(df.QCleaned==qcleaned) & (df.Country==country)].Q.value_counts()

def tab(cpnum=None,cp=None):
    assert cpnum!=None or cp!=None, "pass in at least 1 param."
    if cpnum!=None:
        cp=CPCleanedUnique[cpnum]
    print(cp)
    global df
    filtered_df=df[df['CPCleaned']==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def discardYear(qcleaned, country, year:int=2021):
    global df
    df.loc[(df.QCleaned==qcleaned) & (df.Country==country) & (df.AssesmentDate.str[-4:]==str(year)),"CPCleaned"]="delete"

---

In [14]:
tab(cp='osh management systems')


osh management systems


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
does the employer investigate work-related accidents and indicate the technical recommendations necessary to prevent them?,0,0,0,0,0,122,0,122
does the employer record work-related accidents?,0,0,161,0,0,0,0,161
does the factory have a written osh policy?,0,0,0,0,474,0,0,474
has legal construction permits/structural safety certificate,3486,9266,428,5070,2370,1205,12614,34439
has the employer performed an assessment of general occupational safety and health issues in the factory?,0,502,0,0,0,0,1802,2304
mechanisms/committee for cooperation on osh,0,2503,55,0,0,0,0,2558
All,3486,12271,644,5070,2844,1327,14416,40058


In [15]:
replace(["has the employer done an initial industrial hygiene risk assessment, a risk map, and annual risk assessments?","has the employer performed an assessment of general occupational safety and health issues in the factory?"],"Assess OSH issues")
#combined Nicaragua item with others

In [16]:
replace(["has legal construction permits/structural safety certificate","does the employer have osh technical organizational regulations (tor) that are approved by the ministry of labor?"])
#combined one in Nicaragua with others

In [17]:
df.loc[(df.QCleaned=="does the factory have a valid business license?") & (df.Country=="Jordan"),"QCleaned"]="has legal construction permits/structural safety certificate"
##move one in Jordan to others

In [18]:
replace(["does the employer develop an osh plan annually?","does the employer have an osh license?"],"does the factory have a written osh policy?")
#move one item in Vietnam to others

In [19]:
replace(["does the factory employ the required osh specialists?","has the employer appointed an officer trained in firefighting, rescue and first aid?"])

In [20]:
discard("does the employer have an steam generator license?") #one unique in Nicaragua
discard("does the employer report both accidents and non-occurrences to the ministry of labor?") #one unique item in Nicaragua

In [21]:
replace(["has the employer set up a properly functioning osh collaborators network?"],"does the factory employ the required osh specialists?")
#move one in Vietnam to similar others on osh personnel

In [22]:
df.loc[(df.QCleaned=="does the factory have a valid business license?") & (df.Country=="Bangladesh"),"QCleaned"]="does the factory have a written osh policy?"
##move one in Bangladesh to others

In [23]:
for i in ["does the employer have the required certificates for the installation/ operation of electrical installations and other machines and equipment?",
       "does the employer inspect and measure the environmental conditions in the workplace on a annual basis?",
       "does the employer investigate work-related accidents and indicate the technical recommendations necessary to prevent them?",
       "does the employer regularly inspect and maintain machines, equipment, buildings and stores?",
       "has the employer designated a person exclusively devoted to osh promotion, prevention and protection?",
       "has the employer elaborated an osh training program?",
       "does the factory employ the required osh specialists?"]:
    assign(i)


In [24]:
assign("does the employer comply with requirements on hiv/aids?")

In [25]:
getOriginalQ("mechanisms/committee for cooperation on osh")

has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?    2558
Name: Q, dtype: int64

In [26]:
collapse("mechanisms/committee for cooperation on osh","Cambodia")

has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?    2503
Name: Q, dtype: int64

In [27]:
# df.loc[df.Q=="does the workplace have a functioning hiv/aids committee?", "QCleaned"]="does the workplace have a functioning hiv/aids committee?"

In [28]:
df.to_csv("d://r4.csv",index=False)

In [31]:
# df.to_pickle("d://r4.pkl")
df.to_feather("d://r4.feather")

In [40]:
for i in range(1000000):
    print(i,"\r",end="")